# Lecture 5 Recap
---

## 1. Minimum Variance Frontier
*   Minimum-variance frontier shows the risk-return opportunities available to the investor.
*   Efficient frontier
    *   There is an efficient portfolio for each risk level.
    *   Portfolios on the efficient frontier represent trade-offs in terms of risk and return.



<center>
<img src="../BFIN_Tutorial 3/Picture7.png" width="600"/>


## 2. Capital Allocation: Risk-free and Risky Assets
### Complete portfolio consists of the risky asset and the risk-free asset.
  
  * A portfolio risk closer to the desired level can be achieved by **changing the weights** of risk-free asset and risky assets in the complete portfolio.

$$
      W_{rf} + W_{risky} = 1
$$

$$
      E(r_p) = (1 - W_{risky}) \cdot r_f + W_{risky} \cdot E(r_{risky})
$$

$$
      E(r_p) = r_f + W_{risky} \cdot [E(r_{risky}) - r_f]
$$

$$
      \sigma^2(r_p) = w_{rf}^2 \cdot \sigma^2(r_f) + W_{risky}^2 \cdot \sigma^2(r_{risky}) + 2 \cdot W_{rf}W_{risky}Cov(r_f, r_{risky})
$$

* Since $\sigma^2(r_f) = 0$ and $Cov(r_f, r_{risky}) = 0$,

$$
      \sigma^2(r_p) = W_{risky}^2 \cdot \sigma^2(r_{risky})
$$


<center>
<img src="../BFIN_Tutorial 3/Picture2.png" width="500"/>

- Investors choose a point on the CAL based on their risk preference, using their indifference curves.
- Target: **highest possible utility**.

<center>
<img src="../BFIN_Tutorial 3/Picture3.png" width="700"/>

### Calculating Portfolio Expected Returns Using Standard Deviations
$$
E(r_p) = r_f + \left[ \frac{E(r_{risky}) - r_f}{\sigma(r_{risky})} \right] \cdot \sigma(r_p)
$$

* **Example**: Suppose the risk-free rate is 3.5%, the expected return and the standard deviation of the risky portfolio are 9% and 12%, respectively. What is portfolio's expected return?

## 3. The Investment Decision
- Top-down process with 3 steps:
  
  * Capital allocation: risky portfolio and risk-free asset
  
  * Asset allocation: across broad asset classes
  
  * Security selection: individual assets within an asset class


- **Finding the Optimal Risky Portfolio**
  * Maximizing the slope of the CAL (**Sharpe Ratio**)

  $$
      Sharpe Ratio = \frac {[E(r_{P}) - r_f]}{\sigma(r_{P})}
  $$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown, Math

# --- 1. Define Asset Parameters ---
E_rD, std_D = 0.08, 0.12  # Debt Fund
E_rE, std_E = 0.13, 0.20  # Equity Fund
correlation_DE = 0.30

# --- 2. Core Calculation Functions ---
def portfolio_performance(w_E):
    w_D = 1 - w_E
    E_rp = w_D * E_rD + w_E * E_rE
    cov_DE = correlation_DE * std_D * std_E
    var_p = w_D**2 * std_D**2 + w_E**2 * std_E**2 + 2 * w_D * w_E * cov_DE
    return E_rp, np.sqrt(var_p)

# --- 3. Combined Plotting and Info Function ---
def generate_output(weight_in_equity_pct, rf_rate_pct):
    weight_in_equity = weight_in_equity_pct / 100
    rf_rate = rf_rate_pct / 100
    
    # --- Calculate portfolio values ---
    E_r_selected, std_selected = portfolio_performance(weight_in_equity)
    
    # Find the Optimal Risky Portfolio (Tangency Portfolio)
    weights_E_frontier = np.linspace(0, 1, 100)
    frontier_returns, frontier_stds = np.array([portfolio_performance(w) for w in weights_E_frontier]).T
    sharpe_ratios = (frontier_returns - rf_rate) / frontier_stds
    optimal_idx = np.nanargmax(sharpe_ratios)
    E_r_optimal, std_optimal = frontier_returns[optimal_idx], frontier_stds[optimal_idx]
    sharpe_optimal = (E_r_optimal - rf_rate) / std_optimal
    
    # --- a. Info Panel (Markdown + MathJax) ---
    info_md = f"""
#### Asset Profiles:
- Debt Fund: E(r) = {E_rD:.2%}, σ = {std_D:.2%}
- Equity Fund: E(r) = {E_rE:.2%}, σ = {std_E:.2%}

#### Selected Portfolio ({weight_in_equity_pct:.0f}% Stocks):
- E(rₚ) = {E_r_selected:.2%}
- σₚ = {std_selected:.2%}

#### Optimal Risky Portfolio (P):
- E(rₚ) = {E_r_optimal:.2%}
- σₚ = {std_optimal:.2%}
- Sharpe Ratio = {sharpe_optimal:.2f}

#### Objective:
Maximize the slope of the CAL (Sharpe Ratio):
"""
    with output_info:
        clear_output(wait=True)
        display(Markdown(info_md))
        display(Math(r"S_P = \frac{E(r_p) - r_f}{\sigma_p}"))

    # --- b. Plot Generation ---
    with output_plot:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(7.5, 7.5))
        ax.plot(frontier_stds * 100, frontier_returns * 100, color='deepskyblue', linewidth=2.5)

        sharpe_selected = (E_r_selected - rf_rate) / std_selected if std_selected > 0 else 0
        cal_selected_x = np.array([0, 0.25]); cal_selected_y = rf_rate + sharpe_selected * cal_selected_x
        ax.plot(cal_selected_x * 100, cal_selected_y * 100, color='black', linewidth=1.5)
        ax.scatter(std_selected * 100, E_r_selected * 100, facecolors='none', edgecolors='black', s=100, zorder=5, linewidth=1.5)

        cal_optimal_x = np.array([0, 0.25]); cal_optimal_y = rf_rate + sharpe_optimal * cal_optimal_x
        ax.plot(cal_optimal_x * 100, cal_optimal_y * 100, color='black', linewidth=1.5)
        ax.scatter(std_optimal * 100, E_r_optimal * 100, color='black', s=100, zorder=5)

        points = {'A': portfolio_performance(0.18), 'B': portfolio_performance(0.30), 'E': portfolio_performance(0.82)}
        for label, (ret, std) in points.items():
            ax.scatter(std * 100, ret * 100, facecolors='none', edgecolors='black', s=100, zorder=5, linewidth=1.5)
            ax.text(std * 100 + 0.3, ret * 100, label, fontsize=11, verticalalignment='center')

        ax.text(portfolio_performance(0.18)[1]*100, 8.0, ' D', verticalalignment='center')
        ax.plot([0, 25], [8, 8], 'k--', alpha=0.7); ax.plot([0, 25], [13, 13], 'k--', alpha=0.7)
        ax.set_title('The Investment Decision', fontsize=14)
        ax.set_xlabel('Standard Deviation (%)', fontsize=12); ax.set_ylabel('Expected Return (%)', fontsize=12)
        ax.set_xlim(0, 25); ax.set_ylim(5, 14)
        ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
        ax.tick_params(axis='both', which='major', labelsize=11)
        plt.show()

# --- 4. Interactive Widgets Setup ---
style = {'description_width': 'initial'}
equity_weight_slider = widgets.FloatSlider(value=18.0, min=0, max=100, step=1, description='Weight in Stocks (%):', style=style, continuous_update=True)
rf_rate_slider = widgets.FloatSlider(value=5.0, min=2, max=8, step=0.25, description='Risk-Free Rate (%):', style=style, continuous_update=True)
output_info = widgets.Output(layout=widgets.Layout(overflow='auto'))
output_plot = widgets.Output()
io_link = widgets.interactive_output(generate_output, {'weight_in_equity_pct': equity_weight_slider, 'rf_rate_pct': rf_rate_slider})

# --- 5. Side-by-side Layout ---
header = widgets.HTML("<h3>Finding the Optimal Risky Portfolio</h3>")
controls = widgets.VBox([equity_weight_slider, rf_rate_slider], layout=widgets.Layout(gap='10px'))
left_col = widgets.VBox([header, controls, output_info], layout=widgets.Layout(width='45%', padding='0 12px 0 0'))
right_col = widgets.VBox([output_plot], layout=widgets.Layout(width='55%'))
ui = widgets.HBox([left_col, right_col], layout=widgets.Layout(align_items='flex-start'))

display(ui)
generate_output(equity_weight_slider.value, rf_rate_slider.value)

- **Finding the Optimal Complete Portfolio**  
$$U = E(r) - \frac{1}{2} A \sigma^2$$

---

# Lecture 6 Recap
---


## 1. The Capital Asset Pricing Model (CAPM)

### 1.1. Assumptions
- Investors are mean–variance optimizers with the same horizon, and they can borrow/lend at a risk-free rate.  
- Markets are efficient (market prices react rapidly to new information) and frictionless (no taxes/transaction costs/buying and selling restrictions).  
- All assets are tradable and divisible.  
- Homogeneous expectations: same beliefs about expected returns, variances, and covariances.

### 1.2. Main CAPM Equation

For any security \(i\):

$$
E(R_i) = r_f + \beta_i \left[ E(R_M) - r_f \right]
$$

where beta is

$$
\beta_i = \frac{\text{Cov}(R_i, R_M)}{\text{Var}(R_M)}
= \frac{\sigma_i}{\sigma_M}\rho_{i,M}
$$

- **Only systematic risk (beta)** is priced.  
- Firm-specific risk can be diversified away and **does not affect expected return**.



<center>
<img src="../BFIN_Tutorial 3/Picture4.1.png" width="700"/>

---


## 2. Security Market Line (SML) vs Capital Market Line (CML)

### 2.1. Security Market Line (SML)

Plots **expected return vs. beta**:

$$
E(R_i) = r_f + \beta_i \left[ E(R_M) - r_f \right]
$$

- Securities **above SML**: positive alpha (undervalued).  
- Securities **below SML**: negative alpha (overvalued).

### 2.2. SML vs. CML

| Line | X-axis | Y-axis | Who lies on it? | Use     |
|------|--------|--------|-----------------|--------------|
| **CML** | Total risk $ \sigma\ $ | Expected return | Only risk-free asset and market portfolio | Evaluating portfolios |
| **SML** | Systematic risk $ \beta $ | Expected return | All securities and portfolios | Evaluating individual securities |




<img src="../BFIN_Tutorial 3/Picture5.png" width="550"/>
<img src="../BFIN_Tutorial 3/Picture6.png" width="420"/>
</center>


---


## 3. Cost of Capital & Equity Risk Premium

**Cost of equity** for firm \(i\) under CAPM:

$$
k_e = r_f + \beta_i \big(R_m) - r_f \big)
$$

Long-run US historical averages (approx.):

| Metric                               | Avg. annual return (1928–2024) |
|--------------------------------------|--------------------------------:|
| S&P 500 total return                 | 11.79% |
| 3-month T-bill                       | 3.36%  |
| **Equity risk premium vs T-bill**    | **8.43%** |
| 10-year Treasury bond                | 4.79%  |
| **Equity risk premium vs 10-yr bond**| **7.00%** |

These historical premiums give a **benchmark** for the market risk premium.

---

## 4. Beta Estimation in Practice

Estimate beta using historical regressions of excess returns:

$$
(R_i - r_f) = \alpha_i + \beta_i (R_M - r_f) + e_i
$$

**Standard error of beta:**

$$
\text{S.E.}(\hat{\beta}_i) =
\frac{\sqrt{1 - R^2}}{\sqrt{T}}
\cdot
\frac{\hat{\sigma}_i}{\hat{\sigma}_M}
$$

To improve precision:

- Use a **larger sample** (\(T\)).  
- Use **portfolios** instead of individual stocks.  
- Use an appropriate **return frequency** (e.g., monthly).

---
---




# Questions

### Question 1 – Two-Asset Portfolio with Risk-Free Asset

You are given two risky assets, **P** and **Q**, and a risk-free asset:

| Asset     | Expected Return | Std. Dev. |
|-----------|-----------------|-----------|
| P         | 11%             | 18%       |
| Q         | 16%             | 28%       |
| Risk-Free | 3%              | 0%        |

The correlation between P and Q is **0.25**.

1. Find the **weights, expected return, and standard deviation** of the **Optimal Risky Portfolio (tangency portfolio)** formed by P and Q.  
2. An investor has **risk-aversion** $A = 4$ and can invest in the tangency portfolio and the risk-free asset. What is the **optimal allocation** \( y^\* \) to the risky portfolio?  
3. What are the **expected return** and **standard deviation** of this investor’s **complete portfolio**?

---

### Question 2 – $R^2$ and the Single-Index Model (Conceptual)

A security analyst runs a single-index regression of Stock **D**’s excess returns on the market’s excess returns and obtains an $R^2$ of **0.70**.

1. What does this $R^2$ tell you about the **variability of Stock D’s returns**?  
2. Does a high $R^2$ necessarily imply that Stock D will offer a **higher expected return** under the CAPM? Briefly explain.

---

### Question 3 – Single-Index Model Calculations

You are analyzing Stock **R** with the single-index model. From a regression of R’s **excess returns** on the market’s excess returns, you obtain:

- Beta $beta_R = 1.3$
- Alpha $alpha_R = 1.5\%$ per month  
- $R^2 = 0.36$
- Standard deviation of the market’s excess returns: $\sigma_M = 5\%$ per month  

1. What is the **standard deviation of Stock R’s excess returns**?  
2. What is the **covariance** between Stock R and the market?  
3. What is the **correlation coefficient** between Stock R and the market?

---

### Question 4 – CAPM vs Total Risk

The **CAPM** states that the expected return of a security depends only on its **systematic risk** (beta), not on its total standard deviation. Yet, when constructing an optimal risky portfolio with **Markowitz portfolio theory**, the **full covariance matrix of returns** is required.

Explain how these two ideas are **reconciled**.

---

### Question 5 – Two Stocks with Single-Index Regressions

An analyst estimates the following single-index regressions of excess returns on the market’s excess returns:

| Stock | Alpha \( \alpha \) | Beta \( \beta \) | Residual Std. Dev. \( \sigma_\varepsilon \) |
|-------|--------------------|------------------|----------------------------------------------|
| A     | 1.2%               | 1.1              | 7%                                           |
| B     | –0.4%              | 0.9              | 5%                                           |

The expected excess return on the market is \( E(R_M) = 8\% \) and the standard deviation of the market’s excess returns is \( \sigma_M = 18\% \).

1. What are the **expected excess returns** for Stocks A and B?  
2. What is the **covariance** between the returns of Stocks A and B?  
3. Consider an **equally weighted** portfolio of A and B. What are this portfolio’s:
   - alpha,  
   - beta,  
   - residual standard deviation, and  
   - total standard deviation?

---

### Question 6 – Active Portfolio and Kappa Corp

You are evaluating **Kappa Corp** for potential mispricing using the CAPM. You have:

- Kappa Corp’s beta: $ \beta_K $ = 1.4
- Kappa’s residual standard deviation: $ \sigma_{\varepsilon,K} = 12\% $
- Your forecast for Kappa’s expected return: $ E(R_K) = 18\% $
- Your forecast for the market return: $$ E(R_M) = 11\% $
- Market standard deviation: $ \sigma_M = 19\% $
- Risk-free rate: $ r_f = 3\% $

You plan to form an **optimal active portfolio** by combining Kappa Corp with the passive market index.

1. According to the CAPM, what is the **required rate of return** for Kappa Corp?  
2. What is Kappa’s **alpha** based on your forecasts?  
3. Compute the **optimal weight assigned to Kappa**, $ w_A^* $, in the risky active portfolio of Kappa and the market, using  
   $$
      w_A^* = \frac{\alpha / \sigma_{\varepsilon,K}^2}{\left(E(R_M) - r_f\right)/\sigma_M^2}
   $$  
4. What is the **beta of the new optimal risky portfolio** formed by Kappa and the market?  
5. An analyst claims that because the new portfolio’s beta is greater than 1, its **Sharpe ratio must be lower than the market’s**. Is this correct? Compute the **Sharpe ratio** of the new portfolio to justify your answer.

---

# Solutions

### Solution 1 – Two-Asset Portfolio with Risk-Free Asset

**Given**

- $ E(R_P) = 0.11,\ \sigma_P = 0.18 $
- $ E(R_Q) = 0.16,\ \sigma_Q = 0.28 $
- $ r_f = 0.03 $
- $ \rho_{PQ} = 0.25 $

---

**1. Tangency portfolio**

Covariance:
$$
\text{Cov}(P,Q)
= \rho_{PQ} \sigma_P \sigma_Q
= 0.25 \times 0.18 \times 0.28
= 0.0126
$$

Variance–covariance matrix:
$$
\Sigma =
\begin{bmatrix}
0.18^2 & 0.0126 \\
0.0126 & 0.28^2
\end{bmatrix}
=
\begin{bmatrix}
0.0324 & 0.0126 \\
0.0126 & 0.0784
\end{bmatrix}
$$

Excess mean vector:
$$
\mu - r_f \mathbf =
\begin{bmatrix}
0.11 - 0.03 \\
0.16 - 0.03
\end{bmatrix}
=
\begin{bmatrix}
0.08 \\
0.13
\end{bmatrix}
$$

Unnormalised tangency weights:
$$
w^u = \Sigma^{-1} (\mu - r_f \mathbf )
\approx
\begin{bmatrix}
1.9459 \\
1.3454
\end{bmatrix}
$$

Normalise to sum to 1:
$$
w_P = \frac{1.9459}{1.9459 + 1.3454} \approx 0.591, \qquad
w_Q = 1 - w_P \approx 0.409.
$$

Expected return of tangency portfolio:
$$
E(R_T)
= 0.591 \times 0.11 + 0.409 \times 0.16
\approx 0.1304
= 13.04\%.
$$

Variance and standard deviation:
$$
\sigma_T^2 = w^\top \Sigma w \approx 0.0305,
\qquad
\sigma_T \approx \sqrt{0.0305} \approx 0.1747 = 17.47\%.
$$


---
***NOTE: Computing the Unnormalised Tangency Weights $w^u$ in 5 steps:***

**1. Variance–covariance matrix**

$$
\Sigma =
\begin{bmatrix}
0.18^2 & 0.0126 \\
0.0126 & 0.28^2
\end{bmatrix}
=
\begin{bmatrix}
0.0324 & 0.0126 \\
0.0126 & 0.0784
\end{bmatrix}
$$

**2. Excess mean (risk-premium) vector**

$$
\mu - r_f =
\begin{bmatrix}
0.11 - 0.03 \\
0.16 - 0.03
\end{bmatrix}
=
\begin{bmatrix}
0.08 \\
0.13
\end{bmatrix}
$$


**3. Invert the covariance matrix**

For
$$
\Sigma =
\begin{bmatrix}
a & b\\
c & d
\end{bmatrix},
\quad
\Sigma^{-1} = \frac{1}{ad - bc}
\begin{bmatrix}
d & -b\\
-c & a
\end{bmatrix}
$$

Here:
$$
a = 0.0324, b = 0.0126, c = 0.0126, d = 0.0784
$$

Determinant:
$$
ad - bc = 0.0324 \times 0.0784 - 0.0126 \times 0.0126
\approx 0.0023814
$$

Thus:
$$
\Sigma^{-1}
\approx
\begin{bmatrix}
32.92 & -5.29\\
-5.29 & 13.61
\end{bmatrix}
$$


**4. Unnormalised tangency weights**

$$
w^{u} = \Sigma^{-1}(\mu - r_f)
=
\begin{bmatrix}
32.92 & -5.29\\
-5.29 & 13.61
\end{bmatrix}
\begin{bmatrix}
0.08\\
0.13
\end{bmatrix}
\approx
\begin{bmatrix}
1.9459\\
1.3454
\end{bmatrix}
$$


**5. Normalise to obtain portfolio weights**

$$
w_P = \frac{1.9459}{1.9459 + 1.3454} \approx 0.591,
\qquad
w_Q = 1 - w_P \approx 0.409
$$

So the tangency portfolio invests approximately **59.1% in asset \(P\)** and **40.9% in asset \(Q\)**.

---

**2. Optimal allocation to risky portfolio**

For a mean–variance investor:
$$
  y = \frac{E(R_T) - r_f}{A \sigma_T^2}
  = \frac{0.1304 - 0.03}{4 \times 0.0305}
  \approx 0.823.
$$

So the investor holds **82.3%** in the tangency portfolio and **17.7%** in the risk-free asset.

---

**3. Complete portfolio**

Expected return:
$$
E(R_C)
= r_f + y* \big(E(R_T) - r_f\big)
= 0.03 + 0.823 \times 0.1004
\approx 0.1126
= 11.26\%.
$$

Standard deviation:
$$
\sigma_C = y * \sigma_T
= 0.823 \times 0.1747
\approx 0.1437
= 14.37\%.
$$

---

### Solution 2 – $R^2$ and the Single-Index Model

1. $R^2$ = 0.70 means that **70% of the variance** of Stock D’s **excess returns** is explained by movements in the market (systematic risk). The remaining **30%** is firm-specific (idiosyncratic) risk.

2. Under the **CAPM**, expected return depends on **beta**, not directly on $R^2$ or total variance. A high $R^2$ only indicates that the stock’s returns track the market closely; it does **not** guarantee a higher beta or higher expected return. Therefore, a high $R^2$ does **not** imply a superior or higher-return investment.

---

### Solution 3 – Single-Index Model Calculations

Given:

- $beta_R = 1.3$
- $R^2 = 0.36$
- $sigma_M = 0.05$

---

**1. Standard deviation of Stock R**

In the single-index model:
$$
R^2 = \frac{\beta_R^2 \sigma_M^2}{\sigma_R^2}
\quad\Rightarrow\quad
\sigma_R = \frac{\beta_R \sigma_M}{\sqrt{R^2}}.
$$

So:
$$
\sigma_R
= \frac{1.3 \times 0.05}{\sqrt{0.36}}
= \frac{0.065}{0.6}
\approx 0.1083
= 10.83\% \text{ per month}.
$$

---

**2. Covariance with the market**

$$
\text{Cov}(R,M)
= \beta_R \sigma_M^2
= 1.3 \times 0.05^2
= 1.3 \times 0.0025
= 0.00325.
$$

---

**3. Correlation coefficient**

$$
\rho_{R,M}
= \frac{\text{Cov}(R,M)}{\sigma_R \sigma_M}
= \frac{0.00325}{0.1083 \times 0.05}
\approx 0.60.
$$

(Notice $ \rho_{R,M} = \sqrt{R^2} = 0.6,$ as expected.

---

### Solution 4 – CAPM vs Total Risk

* **Markowitz portfolio theory** helps answer the question: “If I have several risky assets, how do I combine them to get the best trade-off between risk and return?”

    * Return: $E(rp​)=i∑​wi​E(ri​)$

    * Variance: $σ_p^2​=i∑​j∑​w_i*​w_j*​σ_{ij}​ $

    Instead of looking at each stock alone, he looked at all of them together.

* **CAPM**: With many securities, investors can diversify away **idiosyncratic risk**. Thus, all investors hold some combination of the **market portfolio** and the **risk-free asset**.

    *  In that equilibrium, only **systematic risk with the market** (captured by beta) cannot be diversified away. Hence, only beta is **priced** and determines expected returns in the **CAPM**:
    * Return: $E(R_i) = r_f + \beta_i \big(E(R_M) - r_f\big) $

So, total risk and the full covariance matrix matter for **portfolio construction**, while **only beta** matters for **pricing** in the CAPM equilibrium. The two ideas are consistent.

---

### Solution 5 – Two Stocks with Single-Index Regressions

Given:

- Stock A: $\alpha_A = 0.012$,  $\beta_A = 1.1$, $\sigma_{\varepsilon,A} = 0.07$  
- Stock B: $\alpha_B = -0.004$, $\beta_B = 0.9$, $\sigma_{\varepsilon,B} = 0.05$ 
- Market: $E(R_M) = 0.08$, $\sigma_M = 0.18$

---

**1. Expected excess returns**

For stock \( i \):
$$
E(R_i) = \alpha_i + \beta_i E(R_M).
$$

- Stock A:
  $$
  E(R_A) = 0.012 + 1.1 \times 0.08
  = 0.012 + 0.088
  = 0.10
  = 10\%.
  $$

- Stock B:
  $$
  E(R_B) = -0.004 + 0.9 \times 0.08
  = -0.004 + 0.072
  = 0.068
  = 6.8\%.
  $$

---

**2. Covariance between A and B**

Residuals are uncorrelated in the single-index model, so:
$$
\text{Cov}(A,B)
= \beta_A \beta_B \sigma_M^2
= 1.1 \times 0.9 \times 0.18^2
= 0.99 \times 0.0324
\approx 0.03208.
$$

---

**3. Equally weighted portfolio \((w_A = w_B = 0.5)\)**

- **Alpha**:
  $$
  \alpha_P = 0.5 \alpha_A + 0.5 \alpha_B
  = 0.5 \times 0.012 + 0.5 \times (-0.004)
  = 0.004
  = 0.4\%.
  $$

- **Beta**:
  $$
  \beta_P = 0.5 \beta_A + 0.5 \beta_B
  = 0.5 (1.1 + 0.9)
  = 1.0.
  $$

- **Residual variance**:
  $$
  \sigma_{\varepsilon,P}^2
  = 0.5^2 \sigma_{\varepsilon,A}^2
  + 0.5^2 \sigma_{\varepsilon,B}^2
  = 0.25 (0.07^2 + 0.05^2)
  = 0.25 (0.0049 + 0.0025)
  = 0.00185.
  $$

  Residual standard deviation:
  $$
  \sigma_{\varepsilon,P}
  = \sqrt{0.00185}
  \approx 0.0430
  = 4.30\%.
  $$

- **Total variance**:
  $$
  \sigma_P^2
  = \beta_P^2 \sigma_M^2 + \sigma_{\varepsilon,P}^2
  = 1^2 \times 0.18^2 + 0.00185
  = 0.0324 + 0.00185
  = 0.03425.
  $$

  Total standard deviation:
  $$
  \sigma_P
  = \sqrt{0.03425}
  \approx 0.1851
  = 18.51\%.
  $$

---



### Solution 6 – Active Portfolio and Kappa Corp

#### Given

- Kappa beta: $\beta_K$ = 1.4
- Kappa residual st.dev.: $sigma_{\varepsilon,K} = 12%$ = 0.12
- Forecast $E(R_K) = 18\% = 0.18$
- Forecast $E(R_M) = 11\% = 0.11$
- Market st.dev.: $\sigma_M = 19% = 0.19$
- Risk-free rate: $r_f = 3%$ = 0.03

---

### 1. Required rate of return on Kappa under CAPM

Using CAPM:

$$
E(R_K)^{\text{CAPM}} = r_f + \beta_K\big(E(R_M) - r_f\big)
$$

Substitute:

$$
E(R_K)^{\text{CAPM}} 
= 0.03 + 1.4(0.11 - 0.03)
= 0.03 + 1.4 \times 0.08
= 0.03 + 0.112
= 0.142 = 14.2\%.
$$

**Answer (1):** Required return \(= 14.2\%\).

---

### 2. Kappa’s alpha based on the forecasts

Alpha:

$$
\alpha_K 
= E(R_K)^{\text{forecast}} - E(R_K)^{\text{CAPM}}
= 0.18 - 0.142
= 0.038 = 3.8\%.
$$

**Answer (2):** \( \alpha_K = 3.8\% \).

---

### 3. Optimal weight assigned to Kappa in the risky active portfolio

Given formula:

$$
w_A^* 
= \frac{\alpha / \sigma_{\varepsilon,K}^2}
{ \big(E(R_M) - r_f\big)/\sigma_M^2 }.
$$

Compute each term:

$$
\sigma_{\varepsilon,K}^2 = 0.12^2 = 0.0144,
\qquad
\frac{\alpha}{\sigma_{\varepsilon,K}^2}
= \frac{0.038}{0.0144} \approx 2.6389.
$$

$$
\sigma_M^2 = 0.19^2 = 0.0361,
\qquad
\frac{E(R_M) - r_f}{\sigma_M^2}
= \frac{0.11 - 0.03}{0.0361}
= \frac{0.08}{0.0361} \approx 2.2161.
$$

Therefore:

$$
w_A^* = \frac{2.6389}{2.2161} \approx 1.1908.
$$

Interpretation: about **119.1% long Kappa** and **19.1% short the market** in the risky active portfolio.

**Answer (3):** \( w_A^* \approx 1.19 \).

---

### 4. Beta of the new optimal risky portfolio

Let

$$
w_K = w_A^* \approx 1.1908,
\qquad
w_M = 1 - w_K \approx -0.1908.
$$

Portfolio beta:

$$
\beta_P = w_K \beta_K + w_M \beta_M,
\qquad \beta_M = 1.
$$

$$
\beta_P 
= 1.1908 \times 1.4 + (-0.1908) \times 1
= 1.6671 - 0.1908
\approx 1.4763.
$$

**Answer (4):** \( \beta_P \approx 1.48 \).

---

### 5. Sharpe ratio of the new portfolio vs market

#### Market Sharpe ratio

$$
S_M 
= \frac{E(R_M) - r_f}{\sigma_M}
= \frac{0.11 - 0.03}{0.19}
= \frac{0.08}{0.19}
\approx 0.421.
$$

#### Expected return of new portfolio

$$
E(R_P) = w_K E(R_K) + w_M E(R_M)
$$

$$
E(R_P) 
= 1.1908 \times 0.18 + (-0.1908) \times 0.11
\approx 0.2143 - 0.0210
\approx 0.1934 = 19.34\%.
$$

#### Standard deviation of new portfolio

Use CAPM decomposition: portfolio risk comes from **systematic risk** and **residual risk**.

- Portfolio beta $\beta_P \approx 1.4763$
- Market variance $\sigma_M^2 = 0.0361$
- Kappa’s residual st.dev. $\sigma_{\varepsilon,K} = 0.12$
- Residual risk is only from Kappa, scaled by $w_K$

Variance:

$$
\sigma_P^2 
= \beta_P^2 \sigma_M^2 + (w_K \sigma_{\varepsilon,K})^2.
$$

Compute:

$$
\beta_P^2 \sigma_M^2 
\approx (1.4763)^2 \times 0.0361 \approx 0.0786,
$$

$$
(w_K \sigma_{\varepsilon,K})^2
= (1.1908 \times 0.12)^2 
\approx (0.1429)^2 \approx 0.0205.
$$

$$
\sigma_P^2 \approx 0.0786 + 0.0205 = 0.0991
\quad\Rightarrow\quad
\sigma_P \approx \sqrt{0.0991} \approx 0.3148 = 31.48\%.
$$

#### Sharpe ratio of the new portfolio

$$
S_P 
= \frac{E(R_P) - r_f}{\sigma_P}
= \frac{0.1934 - 0.03}{0.3148}
= \frac{0.1634}{0.3148}
\approx 0.519.
$$

#### Comparison

$$
S_P \approx 0.52 > S_M \approx 0.42.
$$

So **even though the new portfolio’s beta is greater than 1, its Sharpe ratio is *higher* than the market’s**, not lower.

**Answer (5):**

- Market Sharpe ratio: $S_M \approx 0.42$
- New portfolio Sharpe ratio: $S_P \approx 0.52$

The analyst’s claim is **incorrect**.
